<hr style="border:2px solid #0176DE"> </hr>
<center><h1 style="color:#173F8A;"> Escuela de Verano en Metodos Iterativos</h1></center> 
<center><h1 style="color:#173F8A;"> EMI 2024 - CMM Chile</h1></center>
<hr style="border:2px solid #0176DE"> </hr>
<h3 style="color:#173F8A;text-align:right;"> Profesores: &nbsp;Nicolás Barnafi<br>Manuel A. Sánchez<br></h3>

<h3 style="color:#03122E;text-align:right;"> 
    Centro de Modelamiento Matematico <br> 
    Instituto de Ingenieria Matematica y Computacional - IMC UC<br>  
</h3>

<hr style="border:2px solid #03122E"> </hr>
<center><h1 style="color:#173F8A;"> Modulo 3: The Conjugate Gradient Method - NGSolve</h1></center> 
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC:
Primaria: 
celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 
Secundaria
gris oscuro: #707070
-->

## TEST 3: Model Problem Linear Elasticity

We will test the algorithm with the solution of the Linear Elasticity problem over a domain $\Omega\subseteq \mathbb R^{3}$
$$ -\nabla \sigma = f, \quad \sigma = \varepsilon (u) = \frac{1}{2}(\nabla u + \nabla u^\top)$$ 
and displacement and traction boundary conditions
\begin{align*}
u & = u_{D}, \quad \text{on } \Gamma_{D} \\
\sigma n & = g, \quad \text{on } \Gamma_{N}
\end{align*}

In [13]:
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
import ngsolve

In [20]:
# Geometry \Omega
box = Box((0,0,0), (3,0.6,1))
box.faces.name="outer"
cyl = sum( [Cylinder((0.5+i,0,0.5), Y, 0.25,0.8) for i in range(3)] )
cyl.faces.name="cyl"
geo = box-cyl
geo.faces.Min(X).name = "fix"
geo.faces.Max(X).name = "force"
DrawGeo(geo);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [21]:
# Generate Mesh
from ngsolve import *
from ngsolve.webgui import Draw
mesh = Mesh(OCCGeometry(geo).GenerateMesh(maxh=0.5)).Curve(3)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [22]:
# Ejemplo de la solucion
# Lame parameters
E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(3)

# Finite element space
fes = VectorH1(mesh, order=3, dirichlet="fix")
u,v = fes.TnT()
gfu = GridFunction(fes)
with TaskManager():
    a = BilinearForm(InnerProduct(Stress(Sym(Grad(u))), Sym(Grad(v))).Compile()*dx)
    pre = Preconditioner(a, "bddc")
    a.Assemble()

g = CF( (1e-3,0,0) )
f = LinearForm(g*v*ds("force")).Assemble()
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs())*f.vec


In [23]:
Draw(5e4*gfu, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene